<a href="https://colab.research.google.com/github/abhibrat/Bigram-Language-Model/blob/main/Trigram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [3]:
chars = list(sorted(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos = {i:s for s,i in stoi.items()}

In [4]:
import torch
import torch.nn.functional as F

In [5]:
xs = []
ys = []
for w in words:
  chars = ['.', '.'] + list(w) + ['.', '.']
  for ch1, ch2, ch3 in zip(chars, chars[1:], chars[2:]):
    idx1 = stoi[ch1]
    idx2 = stoi[ch2]
    idx = idx1*27 + idx2
    # print(ch1,ch2,ch3)
    xs.append(idx)
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('Number of elements: ', num)

Number of elements:  260179


In [9]:
xs

[0, 5, 148, 364, 352]

In [6]:
#Input to the neurons
xenc = F.one_hot(xs, num_classes=729).float()

# Initialize neurons weights
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((729, 27), generator=g, requires_grad=True)

In [21]:
xenc.shape

torch.Size([228146, 729])

In [32]:
# Training
for i in range(20):
  # Forward Pass
  logits = xenc @ W
  counts = logits.exp()
  probs = counts/counts.sum(1, keepdim=True)
  loss = -probs[torch.arange(num), ys].log().mean()
  # if i%10 == 10:
  print(loss.data.item())

  # Backprop
  W.grad = None
  loss.backward()

  # Gradient Descent
  W.data += -1 * W.grad

1.9971420764923096
1.9971396923065186
1.9971373081207275
1.9971351623535156
1.9971327781677246
1.9971305131912231
1.9971281290054321
1.9971258640289307
1.9971234798431396
1.997120976448059
1.9971188306808472
1.9971165657043457
1.9971143007278442
1.9971117973327637
1.9971094131469727
1.9971071481704712
1.9971047639846802
1.9971026182174683
1.9971001148223877
1.9970979690551758


In [31]:
# Generation after training
for i in range(20):
  out = []
  idx = 0
  prev = 0
  while True:
    new_idx = prev*27 + idx
    # print(new_idx)
    x_enc = F.one_hot(torch.tensor(new_idx), num_classes=729).float()
    # print(f'{x_enc=}')
    logits = x_enc @ W
    counts = logits.exp()
    # print(f'{counts=}')
    probs = counts/counts.sum()
    prev = idx
    idx = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[idx])
    if idx == 0:
      break

  print(''.join(out))

an.
madlel.
badaeealee.
talhlsi.
milei.
faxskylia.
atie.
kfemerin.
natijalee.
ayan.
rosh.
leah.
ie.
robi.
leis.
izdvmgjwrqumilla.
avidien.
aman.
telineyangelplsiani.
broni.


In [17]:
W[0]

tensor([-2.9538,  2.1379,  0.9204,  1.0866,  1.1784,  1.0795, -0.2234,  0.2505,
         0.5183,  0.1263,  1.5384,  1.7401,  1.1059,  1.5852,  0.7896, -0.2803,
        -0.0117, -1.7465,  1.1477,  1.3740,  0.9219, -1.8625, -0.3273, -0.5309,
        -1.3615,  0.0265,  0.5794], grad_fn=<SelectBackward0>)